In [5]:
#list = np.arange(5)
#random_element = random.choice(list)

#random_number = random.randint(1, 10)

Given the network topology

We have the number of nodes "numNodes" = len(g) #g is a graph


In [23]:
class CR:
    CRs = []
    sfw = {"high" : 5, "medium" : 4, "low" : 3}    # Security factor weight
    
    def __init__(self, index, s, d, sl, tk, status):     # Constructor : Gets called as soon as an object is declared
        self.index = index
        self.s = s
        self.d = d
        self.sl = sl    # SL can be hign, mediuum, low
        self.tk = tk
        self.status = status    # Status can be Allocatted, blocked, waiting

    def update_status(self, status) :    # A method to update the status
        status_list = ["allocated", "blocked"]
        if status in status_list :
            self.status = status
        else:
            print("\n Invalid Status")

    def k_sp(self, g, k = 1):
        try:
            # Find the K-shortest paths
            k_shortest_paths = list(nx.shortest_simple_paths(g, source = self.s, target = self.d, weight='weight'))

            # Retrieve path costs
            path_costs = [sum(g[s][d]['weight'] for s, d in zip(path, path[1:])) for path in k_shortest_paths]
        
           # Return both paths and corresponding path costs
            return list(zip(k_shortest_paths, path_costs))[:k]
            
        except nx.NetworkXNoPath:
            # Handle the case where no path exists
            return []
    
    def shortest_path(self, g): # Currently limiting to shortest path instead to k-shortest paths
        return nx.shortest_path(g, source = self.s, target = self.d)
        
    '''
    def allocate_resources(self, path):
        #path = [0, 1, 5, 2]
        # Assuming We have already checked for the K-sp and used FF approach to select this path
        # Selecting this path rwequired checking all the constrains. Can be incorporated in resources_available

        for i in range(len(path)):
            link = Network.links[i, i+1]
            link.allocate_resources(self.sl)    # Creating another method in class "Links" to allocate link resources
            # Allocate wavelength based on sl
            # Select the ts based on constraints
            
    '''

    
    def display_info(self):
        print(f"CR: Index : {self.index}, Source Node ={self.s}, Destination Node={self.d}, Security Level={self.sl}, status={self.status}")
 
#######################################################################################################################################################
    
    @classmethod
    def generate_crs(cls, numCR = 1):            # cls = class, it's the same as self, except unlike __init__, it's not limited to an instance
        CRs = []
        sl_values = ["high", "medium", "low"]
        #n = math.ceil(numCR/3)    # Shall I use math.floor?
        n = math.floor(numCR/3)
        uniform_sample = n*sl_values
        
        for i in range(numCR - 3*n):    # the range would be either 1 or 2. i can either be 0 or 0 & 1
             uniform_sample.append(random.choice(sl_values))

        #print(f"Uniform Sample for {numCR} : {uniform_sample}")
        
        for i in range(numCR):
            # Generating source, destination) pair
            #(s, d) =  random_nodes(numCR)
            s = random.randint(1, numNodes)
            d = random.randint(1, numNodes)   

            while s == d :
                d = random.randint(1, numNodes)  

            # Generating sl            
            sl = random.choice(uniform_sample)    # Randomly selecting from a uniform sample; although it will sometimes generate (4, 4, 2)
            uniform_sample.remove(sl)  

            # will add the logic when i come across it
            tk = 1
            
            # Giving initial status
            status = "initialized"

            cr = cls(i, s, d, sl, tk, status)
            CRs.append(cr)
        
        cls.CRs = CRs
        return CRs

    @classmethod
    def create_priority_queue(cls, queues = 'PQ'):
    #Priority Queue
        CR_1 = []
        CR_0 = []
        CR_minus1 = []

        for cr in cls.CRs :
            if cr.sl == "high":
                CR_1.append(cr)
            elif cr.sl == "medium":
                CR_0.append(cr)
            else:
                CR_minus1.append(cr)

        PQ = CR_1 + CR_0 + CR_minus1

        if queues == 'all':
            return PQ, CR_1, CR_0, CR_minus1
        else:
            return PQ

    @classmethod
    def SRCR(cls, CRs):

        num_allocated_crs = sum(1 for cr in CRs if cr.status == "allocated" )
        #print("num_allocated_crs : ", num_allocated_crs)
        
        SRCR = num_allocated_crs/len(CRs)
        print("The Success rate of connection requests(SRCR) is : ", SRCR)
        
        return SRCR

    @classmethod
    def NSP(cls, CRs):

        nsp = [0, 0, 0, 0]
        for i, priority in enumerate(["high", "medium", "low"]) :
            nsp[i+1] = sum(cls.sfw[priority] for cr in CRs if cr.status == "allocated" and cr.sl == priority)
        
        nsp[0] = sum(nsp[i+1] for i in range(3))
        #nsp = sum(cls.sfw[cr.sl] for cr in CRs if cr.status == "allocated")

        '''
        nsp = 0
        for cr in CRs:
            if cr.status == "allocated":
                nsp += cls.sfw[cr.sl]
        '''
        
        print("\n The NSP of the network is : ", nsp[0])
        return nsp
    
    @classmethod
    def display_all(cls, CRs):
        for cr in CRs:
            cr.display_info()
            

The above class CR has a constructor to initialize the values to its object. It has a class method "generate_crs" which takes the numCR parameter to generate that many CRs. the (s, d) pair is selected randomly. The sl is derived from a uniform list; and is allocated randomly to individual CR. 